In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split




In [105]:
df = pd.read_csv('../Datasets/cases/Sonar/Sonar.csv')
X = df.drop('Class', axis=1)
y = df['Class']

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25, stratify=y)
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,0.3108,0.2933,0.2275,0.0994,0.1801,0.2200,0.2732,0.2862,0.2034,0.1740,0.4130,0.6879,0.8120,0.8453,0.8919,0.9300,0.9987,1.0000,0.8104,0.6199,0.6041,0.5547,0.4160,0.1472,0.0849,0.0608,0.0969,0.1411,0.1676,0.1200,0.1201,0.1036,0.1977,0.1339,0.0902,0.1085,0.1521,0.1363,0.0858,0.0290,0.0203,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,0.3085,0.3425,0.2990,0.1402,0.1235,0.1534,0.1901,0.2429,0.2120,0.2395,0.3272,0.5949,0.8302,0.9045,0.9888,0.9912,0.9448,1.0000,0.9092,0.7412,0.7691,0.7117,0.5304,0.2131,0.0928,0.1297,0.1159,0.1226,0.1768,0.0345,0.1562,0.0824,0.1149,0.1694,0.0954,0.0080,0.0790,0.1255,0.0647,0.0179,0.0051,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,0.2716,0.2374,0.1878,0.0983,0.0683,0.1503,0.1723,0.2339,0.1962,0.1395,0.3164,0.5888,0.7631,0.8473,0.9424,0.9986,0.9699,1.0000,0.8630,0.6979,0.7717,0.7305,0.5197,0.1786,0.1098,0.1446,0.1066,0.1440,0.1929,0.0325,0.1490,0.0328,0.0537,0.1309,0.0910,0.0757,0.1059,0.1005,0.0535,0.0235,0.0155,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,0.2898,0.2812,0.1578,0.0273,0.0673,0.1444,

In [103]:
dtc = DecisionTreeClassifier(random_state=6969)
bagging = BaggingClassifier(random_state=6969 ,estimator=dtc, n_estimators=10, max_samples=100)
bagging.fit(X_train, y_train)
y_pred = bagging.predict(X_test)


In [104]:
print(f1_score(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(log_loss(y_test, y_pred))
print(classification_report(y_test,y_pred))

0.5777777777777777
0.6984126984126984
10.870308164971838
              precision    recall  f1-score   support

           0       0.66      0.91      0.77        34
           1       0.81      0.45      0.58        29

    accuracy                           0.70        63
   macro avg       0.74      0.68      0.67        63
weighted avg       0.73      0.70      0.68        63



USING LOGISTIC REGRESSION

In [91]:
from sklearn.linear_model import LogisticRegression


In [92]:
lr = LogisticRegression(C=0.5)
bagging = BaggingClassifier(random_state=25, estimator=lr, n_estimators=10)

bagging.fit(X_train,y_train)
y_pred = bagging.predict(X_test)

In [93]:
print(f1_score(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(log_loss(y_test, y_pred))
print(classification_report(y_test,y_pred))

0.7169811320754716
0.7619047619047619
8.581822235504085
              precision    recall  f1-score   support

           0       0.74      0.85      0.79        34
           1       0.79      0.66      0.72        29

    accuracy                           0.76        63
   macro avg       0.77      0.75      0.76        63
weighted avg       0.77      0.76      0.76        63



# Using different estimators

In [108]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from tqdm import tqdm

lr = LogisticRegression()
knn = KNeighborsClassifier(n_neighbors=3)
svc1 = SVC(kernel='linear', C=0.5)
svc2 = SVC(kernel='rbf', gamma=0.5, C=0.5)

est_list = [lr, knn, dtc, svc1, svc2]
n_est = [10, 15, 25, 50]
scores = []

for e in tqdm(est_list):
    for n in n_est:
        bagg = BaggingClassifier(estimator=e, n_estimators=n, random_state=25, bootstrap=False, max_samples=100)
        bagg.fit(X_train, y_train)
        y_pred_proba = bagg.predict_proba(X_test)
        y_pred = bagg.predict(X_test)

        scores.append([e,n,log_loss(y_test,y_pred_proba)])

scores = pd.DataFrame(scores, columns=['Estimator name', 'Sample size', 'Log loss'])
scores.sort_values('Log loss')

100%|██████████| 5/5 [00:04<00:00,  1.16it/s]


,Estimator name,Sample size,Log loss
7,KNeighborsClassifier(n_neighbors=3),50,0.420693
6,KNeighborsClassifier(n_neighbors=3),25,0.428393
5,KNeighborsClassifier(n_neighbors=3),15,0.433164
4,KNeighborsClassifier(n_neighbors=3),10,0.466018
11,DecisionTreeClassifier(random_state=6969),50,0.475995
3,LogisticRegression(),50,0.537733
2,LogisticRegression(),25,0.539222
1,LogisticRegression(),15,0.545824
10,DecisionTreeClassifier(random_state=6969),25,0.548430
0,LogisticRegression(),10,0.548876
